For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

3. To create the above dataframe:

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
4. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

In [118]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#conda install -c conda-forge geopy   # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [119]:
url="http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [120]:
#The pd.read_html will return a list of dataframe which the actual datafram is df[0]
df = pd.read_html(url,match="Postcode")


In [121]:
#make the not assign value to np.nan
data = df[0]
data['Borough'].replace("Not assigned",np.nan, inplace=True)
data.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [122]:
#reset the index 
data = data.dropna(subset=['Borough'])
data.reset_index(drop=True,inplace=True)
data.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [124]:
data['Neighbourhood'].replace("Not assigned",np.nan,inplace=True)
data['Neighbourhood'].fillna(data['Borough'],inplace=True)
data.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [125]:
data.dtypes

Postcode         object
Borough          object
Neighbourhood    object
dtype: object

In [31]:
#data['Neighbourhood'] = data.groupby("Postcode")['Neighbourhood'].transform(lambda x: ','.join(x))

/Users/nelson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
#data.drop_duplicates(inplace=True)



/Users/nelson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [126]:
data.head(20)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [127]:
print(data.shape)


(211, 3)


In [128]:
url = "http://cocl.us/Geospatial_data"
corr = pd.read_csv(url)
corr.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [129]:
corr.rename(columns = {"Postal Code":"Postcode"},inplace = True)
#corr.head(10)
place = pd.merge(data, corr, on='Postcode')
place.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
5,M6A,North York,Lawrence Manor,43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,Rouge,43.806686,-79.194353
9,M1B,Scarborough,Malvern,43.806686,-79.194353


In [130]:
Toronto_data = place[place['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568
9,M5H,Downtown Toronto,King,43.650571,-79.384568


In [131]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer") #Nominatim requires this value to be set to your application name. The goal is to be able to limit the number of requests per application.
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [132]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map (neighbourhood)

for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [133]:
CLIENT_ID = 'D3NYWDCSVAIPI3X4L3PG5VUPLBYS0S0R3R1OGR31AXUY3OQS' # your Foursquare ID
CLIENT_SECRET = 'HNJ5K4THVJHCPB0PIUOYFFQEMEOMQNIX4U0BAP5WSAGUMKZY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30 #how many value would like to return
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D3NYWDCSVAIPI3X4L3PG5VUPLBYS0S0R3R1OGR31AXUY3OQS
CLIENT_SECRET:HNJ5K4THVJHCPB0PIUOYFFQEMEOMQNIX4U0BAP5WSAGUMKZY


In [134]:
#Get the first neighborhood 
neighborhood_latitude = Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.6542599, -79.3606359.


In [135]:
# explore the venues near the neighborhood (through URI)
radius = 200
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=D3NYWDCSVAIPI3X4L3PG5VUPLBYS0S0R3R1OGR31AXUY3OQS&client_secret=HNJ5K4THVJHCPB0PIUOYFFQEMEOMQNIX4U0BAP5WSAGUMKZY&ll=43.6542599,-79.3606359&v=20180605&&radius=200&limit=30'

In [136]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5db29caadb1d81002c315db0'},
 'response': {'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 43.6560599018, 'lng': -79.35815270005945},
   'sw': {'lat': 43.6524598982, 'lng': -79.36311909994056}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.653446723052674,
          'lng': -79.3620167174383}],
        'distance': 143,
       

In [137]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [138]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON #Normalize semi-structured JSON data into a flat table.

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Body Blitz Spa East,Spa,43.654735,-79.359874
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,The Extension Room,Gym / Fitness Center,43.653313,-79.359725


In [139]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


In [140]:
#loop
def getNearbyVenues(names, latitudes, longitudes, radius=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [141]:
# type your answer here

manhattan_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )




Harbourfront
Regent Park
Ryerson
Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
Design Exchange
Toronto Dominion Centre
Commerce Court
Victoria Hotel
Harbord
University of Toronto
Chinatown
Grange Park
Kensington Market
CN Tower
Bathurst Quay
Island airport
Harbourfront West
King and Spadina
Railway Lands
South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
St. James Town
First Canadian Place
Underground city
Church and Wellesley


In [142]:
Toronto_venues=manhattan_venues
print(Toronto_venues.shape)
Toronto_venues.head(50)

(162, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
1,Harbourfront,43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
2,Harbourfront,43.654260,-79.360636,Henrietta Lane,43.653911,-79.361395,Cocktail Bar
3,Regent Park,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park,43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
5,Regent Park,43.654260,-79.360636,Henrietta Lane,43.653911,-79.361395,Cocktail Bar
6,Ryerson,43.657162,-79.378937,Ryerson Image Centre,43.657523,-79.379460,Art Gallery
7,Ryerson,43.657162,-79.378937,Balzac's Coffee,43.657854,-79.379200,Coffee Shop
8,Garden District,43.657162,-79.378937,Ryerson Image Centre,43.657523,-79.379460,Art Gallery
9,Garden District,43.657162,-79.378937,Balzac's Coffee,43.657854,-79.379200,Coffee Shop


In [143]:
#used to return how many item in one neighborhood
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,11,11,11,11,11,11
Bathurst Quay,1,1,1,1,1,1
CN Tower,1,1,1,1,1,1
Cabbagetown,1,1,1,1,1,1
Central Bay Street,4,4,4,4,4,4
Chinatown,4,4,4,4,4,4
Christie,1,1,1,1,1,1
Commerce Court,16,16,16,16,16,16
Design Exchange,7,7,7,7,7,7


In [144]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 45 uniques categories.


In [145]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1]) #-1 is from the back
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head(20)

,Neighborhood,American Restaurant,Art Gallery,Bakery,Bar,Bookstore,Breakfast Spot,Building,Burger Joint,Burrito Place,Café,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Deli / Bodega,Dessert Shop,Farmers Market,Fast Food Restaurant,Food Court,Gastropub,Gluten-free Restaurant,Greek Restaurant,Gym,Gym / Fitness Center,Hostel,Hotel,Italian Restaurant,Japanese Restaurant,Liquor Store,Nightclub,Park,Performing Arts Venue,Pharmacy,Pub,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Soup Place,Spa,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Vegetarian / Vegan Restaurant
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Regent Park,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Regent Park,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Ryerson,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Ryerson,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Garden District,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Garden District,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [146]:
Toronto_onehot.shape

(162, 46)

In [147]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,American Restaurant,Art Gallery,Bakery,Bar,Bookstore,Breakfast Spot,Building,Burger Joint,Burrito Place,Café,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Deli / Bodega,Dessert Shop,Farmers Market,Fast Food Restaurant,Food Court,Gastropub,Gluten-free Restaurant,Greek Restaurant,Gym,Gym / Fitness Center,Hostel,Hotel,Italian Restaurant,Japanese Restaurant,Liquor Store,Nightclub,Park,Performing Arts Venue,Pharmacy,Pub,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Soup Place,Spa,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Vegetarian / Vegan Restaurant
0,Adelaide,0.0000,0.0000,0.000000,0.090909,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.090909,0.0,0.090909,0.000000,0.090909,0.00,0.0000,0.090909,0.000000,0.000000,0.090909,0.000000,0.000000,0.000,0.000000,0.000000,0.090909,0.00,0.0,0.0,0.000,0.00,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000000,0.090909,0.090909,0.090909,0.000000,0.090909
1,Bathurst Quay,0.0000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.0,1.000,0.00,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,CN Tower,0.0000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.0,1.000,0.00,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Cabbagetown,0.0000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,1.000000,0.000000,0.00,0.0,0.0,0.000,0.00,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.0000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.500000,0.0,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.25,0.000000,0.000000,0.000000,0.2500,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Chinatown,0.0000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.250000,0.250000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.25,0.0,0.0,0.000,0.00,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Christie,0.0000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.00,1.0,0.0,0.000,0.00,0.000000,0.000000,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Commerce Court,0.0625,0.0625,0.062500,0.000000,0.0625,0.000000,0.000000,0.000000,0.0625,0.062500,0.000000,0.062500,0.0,0.000000,0.062500,0.000000,0.00,0.0625,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.00,0.062500,0.000000,0.062500,0.0625,0.0625,0.0625,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000
8,Design Exchange,0.0000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.428571,0.0,0.000000,0.142857,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000,0.00,0.000000,0.285714,0.000000,0.0000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,First Canadian Place,0.0000,0.0000,0.090909,0.000000,0.0000,0.000000,0.

In [148]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    #DF.T. = This function  transpose index and columns.
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                           venue  freq
0  Vegetarian / Vegan Restaurant  0.09
1                    Coffee Shop  0.09
2               Greek Restaurant  0.09
3                     Food Court  0.09
4                   Dessert Shop  0.09


----Bathurst Quay----
                   venue  freq
0  Performing Arts Venue   1.0
1    American Restaurant   0.0
2   Gym / Fitness Center   0.0
3                  Hotel   0.0
4     Italian Restaurant   0.0


----CN Tower----
                   venue  freq
0  Performing Arts Venue   1.0
1    American Restaurant   0.0
2   Gym / Fitness Center   0.0
3                  Hotel   0.0
4     Italian Restaurant   0.0


----Cabbagetown----
                  venue  freq
0    Italian Restaurant   1.0
1   American Restaurant   0.0
2  Gym / Fitness Center   0.0
3                 Hotel   0.0
4   Japanese Restaurant   0.0


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.50
1  Sandwich Place  0.25
2        Pharmacy  0.25
3      

In [159]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [160]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Adelaide,Vegetarian / Vegan Restaurant,Bar,Greek Restaurant,Dessert Shop,Concert Hall
1,Bathurst Quay,Performing Arts Venue,Vegetarian / Vegan Restaurant,Coffee Shop,Gastropub,Food Court
2,CN Tower,Performing Arts Venue,Vegetarian / Vegan Restaurant,Coffee Shop,Gastropub,Food Court
3,Cabbagetown,Italian Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Gastropub,Food Court
4,Central Bay Street,Coffee Shop,Pharmacy,Sandwich Place,Vegetarian / Vegan Restaurant,Food Court
5,Chinatown,Cocktail Bar,Farmers Market,Liquor Store,Café,Coffee Shop
6,Christie,Nightclub,Vegetarian / Vegan Restaurant,Coffee Shop,Gastropub,Food Court
7,Commerce Court,Gym,Pub,Art Gallery,Bakery,Bookstore
8,Design Exchange,Coffee Shop,Restaurant,Gym,Deli / Bodega,Cocktail Bar
9,First Canadian Place,Gastropub,Deli / Bodega,Thai Restaurant,Bakery,Gym / Fitness Center


In [181]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering #fit the features
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 2, 0, 2, 4, 2, 0, 2], dtype=int32)

In [183]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label3', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Label3,Cluster Label2,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,2.0,2.0,2.0,2.0,Spa,Breakfast Spot,Cocktail Bar,Vegetarian / Vegan Restaurant,Coffee Shop
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,2.0,2.0,2.0,2.0,Spa,Breakfast Spot,Cocktail Bar,Vegetarian / Vegan Restaurant,Coffee Shop
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0.0,0.0,0.0,0.0,Coffee Shop,Art Gallery,Vegetarian / Vegan Restaurant,Gastropub,Food Court
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0.0,0.0,0.0,0.0,Coffee Shop,Art Gallery,Vegetarian / Vegan Restaurant,Gastropub,Food Court
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2.0,2.0,2.0,2.0,Italian Restaurant,Japanese Restaurant,Coffee Shop,Performing Arts Venue,Hostel


In [200]:

#Toronto_merged.drop(5,inplace=True)
Toronto_merged.drop(36,inplace=True)
Toronto_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Label3,Cluster Label2,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,2.0,2.0,2.0,2.0,Spa,Breakfast Spot,Cocktail Bar,Vegetarian / Vegan Restaurant,Coffee Shop
1,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,2.0,2.0,2.0,2.0,Spa,Breakfast Spot,Cocktail Bar,Vegetarian / Vegan Restaurant,Coffee Shop
2,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937,0.0,0.0,0.0,0.0,Coffee Shop,Art Gallery,Vegetarian / Vegan Restaurant,Gastropub,Food Court
3,M5B,Downtown Toronto,Garden District,43.657162,-79.378937,0.0,0.0,0.0,0.0,Coffee Shop,Art Gallery,Vegetarian / Vegan Restaurant,Gastropub,Food Court
4,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2.0,2.0,2.0,2.0,Italian Restaurant,Japanese Restaurant,Coffee Shop,Performing Arts Venue,Hostel
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0.0,0.0,0.0,0.0,Coffee Shop,Pharmacy,Sandwich Place,Vegetarian / Vegan Restaurant,Food Court
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4.0,4.0,4.0,4.0,Nightclub,Vegetarian / Vegan Restaurant,Coffee Shop,Gastropub,Food Court
8,M5H,Downtown Toronto,Adelaide,43.650571,-79.384568,2.0,2.0,2.0,2.0,Vegetarian / Vegan Restaurant,Bar,Greek Restaurant,Dessert Shop,Concert Hall
9,M5H,Downtown Toronto,King,43.650571,-79.384568,2.0,2.0,2.0,2.0,Vegetarian / Vegan Restaurant,Bar,Greek Restaurant,Dessert Shop,Concert Hall
10,M5H,Downtown Toronto,Richmond,43.650571,-79.384568,2.0,2.0,2.0,2.0,Vegetarian / Vegan Restaurant,Bar,Greek Restaurant,Dessert Shop,Concert Hall


In [201]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Label3']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters